# CUSTOM CHATTY-BOT 
## T6 - Artificial Inteligence

### S.Chung, G. Sayas
##### "This bot has a lot to say whitin it's "brain" data, I'm not gonna lie" ~ G. Sayas 

In [1]:
import openai
import pandas as pd
import numpy as np
from dateutil.parser import parse
from openai.embeddings_utils import get_embedding, distances_from_embeddings
import tiktoken

In [8]:
openai.api_key = "sk-proj-1cuLbb74Y07sdCroW71rT3BlbkFJjWcR7r5IxYwskBDrM1If"

In [6]:
#Using the previously created dataset for Q&A's

df_quest= pd.read_csv('questions.csv')
df= pd.read_csv('responses.csv')

,text
0,Who is at risk for Lymphocytic Choriomeningiti...
1,What are the symptoms of Lymphocytic Choriomen...
2,Who is at risk for Lymphocytic Choriomeningiti...
3,How to diagnose Lymphocytic Choriomeningitis (...
4,What are the treatments for Lymphocytic Chorio...
...,...
2043,How to diagnose Primary Hyperparathyroidism
2044,What are the treatments for Primary Hyperparat...
2045,What to do for Primary Hyperparathyroidism
2046,What to do for Primary Hyperparathyroidism


In [10]:
#We create the embeddings for all the possible responses in our dataset with a text component

batch_size = 100
embeddings = []

for i in range(0, len(df), batch_size):
    
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine="text-embedding-ada-002"
    )
    
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df.to_csv("embeddings_txt.csv")

#### Ok, now let's create and interface with all of the features explored in our SampleBot

In [34]:
df= pd.read_csv("embeddings_txt.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)
df

,text,embeddings
0,LCMV infections can occur after exposure to fr...,"[0.00632076058536768, -0.012379089370369911, 0..."
1,LCMV is most commonly recognized as causing ne...,"[-0.026089824736118317, -0.004323475528508425,..."
2,Individuals of all ages who come into contact ...,"[0.010534269735217094, -0.012414691969752312, ..."
3,"During the first phase of the disease, the mos...","[-0.0021115741692483425, -0.004429312888532877..."
4,"Aseptic meningitis, encephalitis, or meningoen...","[-0.019981464371085167, 0.015375477261841297, ..."
...,...,...
2043,Once the diagnosis of primary hyperparathyroid...,"[0.011106631718575954, 0.013886476866900921, 0..."
2044,Surgery Surgery to remove the o...,"[0.0014901341637596488, 0.014967930503189564, ..."
2045,"Eating, diet, and nutrition have not been show...","[0.01875200867652893, -0.0011456005740910769, ..."
2046,Primary hyperparathyroidism is a disorder of t...,"[0.01327613927423954, 0.0026520818937569857, 0..."


In [35]:
#Let's create a copy of our dataframe to provide some organization in our lives, and to bettern the context according to the questions: 

def get_rows_sorted_by_relevance(question, df):
    
    question_embeddings = get_embedding(question, engine='text-embedding-ada-002')
    
    df_copy = df.copy()

    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    df_copy.sort_values("distances", ascending=True, inplace=True)
    
    return df_copy

In [50]:
#To create a prompt

max_token_count = 3000

def create_prompt(question, df, max_token_count):
    
    tokenizer = tiktoken.get_encoding("cl100k_base")
   
    prompt_template = """
    
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    context = []
    for text in get_rows_sorted_by_relevance(question, df)['text'].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format('\n\n###\n\n'.join(context), question)

In [47]:
#Gathering of a second question to prompt with

question_2= df_quest['text'][10]
question_2

'How to prevent Parasites - Cysticercosis'

In [48]:
#Checking if the context is related to the question 
get_rows_sorted_by_relevance(question_2, df)

,text,embeddings,distances
10,"To prevent cysticercosis, the following precau...","[0.028971197083592415, 0.01242285966873169, 0....",0.104124
191,Cystic echinococcosis is controlled by prevent...,"[0.003244369523599744, 0.003403139766305685, 0...",0.130475
6,Cysticercosis is an infection caused by the la...,"[0.016199056059122086, -0.0011663701152428985,...",0.134553
9,Some people with cysticercosis do not need to ...,"[0.016014227643609047, 0.01588621735572815, 0....",0.137043
8,"If you think that you may have cysticercosis, ...","[0.0058918604627251625, 0.01647929660975933, 0...",0.142396
...,...,...,...
1138,"The term ""pinched nerve"" is a colloquial term ...","[-0.012932252138853073, -0.0022062603384256363...",0.332825
1230,Dysgraphia is a neurological disorder characte...,"[-0.017921214923262596, 0.01582418940961361, 0...",0.334851
1913,Diabetes management and treatment is expensive...,"[0.01575271040201187, -0.020176930353045464, 0...",0.337901
1900,Blood pressure is the force of blood flow insi...,"[-0.013363348320126534, 0.01590992510318756, 0...",0.341896


In [54]:
print(create_prompt('How to prevent Parasites - Cysticercosis', df, 2000))


    
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

To prevent cysticercosis, the following precautions should be taken:         - Wash your hands with soap and warm water after using the toilet, changing diapers, and before handling food     - Teach children the importance of washing hands to prevent infection     - Wash and peel all raw vegetables and fruits before eating     - Use good food and water safety practices while traveling in developing countries such as:              - Drink only bottled or boiled (1 minute) water or carbonated (bubbly) drinks in cans or bottles       - Filter unsafe water through an "absolute 1 micron or less" filter AND dissolve iodine tablets in the filtered water; "absolute 1 micron" filters can be found in camping and outdoor supply stores              More on: Handwashing    More on: Food and Water Safety

###

Cystic echinococcosis is controlled by preventi

In [52]:
#To answer a question

def answer_question(question, df, max_prompt_tokens=1800, max_answer_tokens=600):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, say "I don't know"
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model='gpt-3.5-turbo-instruct',
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

In [55]:
custom_answer = answer_question('How to prevent Parasites - Cysticercosis', df)
print(custom_answer)

Wash your hands with soap and warm water after handling dogs, and before handling food.
